In [7]:
import requests
import pandas as pd
from io import StringIO, BytesIO
import numpy as np
from datetime import datetime
import re

def fetch_gsheet(sheet, gid, opt, format_type):
    try:
        url = f"https://docs.google.com/spreadsheets/d/{sheet}/export?format={format_type}&gid={gid}"
        resp = requests.get(url)
        resp.raise_for_status()
        if opt == 1 and format_type == 'csv':
            data = StringIO(resp.text)
            df = pd.read_csv(data, skiprows=1)
            df.columns = df.columns.str.strip()
            df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
            rename_map = {
                'Handle By SA/EC (Who FU in Center)': 'Handled By',
                'Received by Who': 'Handled By'
            }
            df = df.rename(columns={col: rename_map.get(col, col) for col in df.columns})
            df = df.loc[:, df.columns.str.contains('Date First Chat|CENTER|Pemberi Referral \(no HP\)|Handled By', regex=True)]
            df = clean_data(df)

        
        elif opt == 2 and format_type == 'xlsx':
            data = BytesIO(resp.content)
            df = pd.read_excel(data, engine='openpyxl', skiprows=1)
            df.columns = df.columns.str.strip()
            df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
            rename_map = {
                'Handle By SA/EC (Who FU in Center)': 'Handled By',
                'Received by Who': 'Handled By'
            }
            df = df.rename(columns={col: rename_map.get(col, col) for col in df.columns})
            df = df.loc[:, df.columns.str.contains('Date First Chat|CENTER|Pemberi Referral \(no HP\)|Handled By', regex=True)]
            df = clean_data(df)
        
        else:
            raise ValueError("opt and format_type must match, '1' for CSV, '2' for XLSX.")

        return df

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

def clean_data(df):
    df.columns = df.columns.str.strip().str.replace('\n', ' ')
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    df = df[~df.astype(str).apply(lambda x: x.str.contains('=ARRAYFORMULA', case=False, na=False)).any(axis=1)]
    df = clean_date_column(df, 'Date First Chat')
    df = df.dropna(how='all')
    return df

def query_issues(df):
    df = clean_data(df)
    df['Date First Chat'] = pd.to_datetime(df['Date First Chat'], errors='coerce')
    cutoff_date = pd.Timestamp('2025-07-01')
    problem_rows = df[
        df['Pemberi Referral (no HP)'].isna() | (df['Pemberi Referral (no HP)'] == '62') &
        (df['Date First Chat'] >= cutoff_date)
    ]
    problem_rows = drop_nat_dates(problem_rows, 'Date First Chat')
    return problem_rows

def invert_excel(dfs_dict, path):
    with pd.ExcelWriter(path, engine='openpyxl') as writer:
        for sheet_name, df in dfs_dict.items():
            result_df = query_issues(df)
            result_df.to_excel(writer, sheet_name=sheet_name, index=False)
    return path

def clean_date_column(df, col_name):
    df[col_name] = (
        df[col_name]
        .astype(str)
        .str.strip()
        .replace({'': None, 'nan': None, 'NaT': None})
    )

    current_year = datetime.now().year

    def normalize_date(value):
        if value is None:
            return None
        
        if re.match(r'^\d{1,2}\s*[A-Za-z]+$', value):
            value = f"{value} {current_year}"
    
        for fmt in ("%d %b %Y", "%d %B %Y", "%B %d, %Y", "%b %d, %Y"):
            try:
                return datetime.strptime(value, fmt).strftime("%d-%m-%Y")
            except Exception:
                continue
        
        return str(value)

    df[col_name] = df[col_name].apply(normalize_date)
    return df

def drop_nat_dates(df, date_col):
    df = df[df[date_col].notna()]
    return df.reset_index(drop=True)

In [8]:
sheet = "1GiHAMx-K2APmNeMSq6TDgz3s8GdStWFHFYgbK9igFno"
gid = {
    'BSD' : '0',
    'TJD' : '813200714',
    'BGR' : '932839478',
    'DPK' : '412631322',
    'KLM' : '1467331128',
    'KGD' : '998827607',
    'BKP' : '1894337291',
    'BHI' : '377073391',
    'BTR' : '846251133',
    'TCT' : '2001460913',
    'PJT' : '2067417063',
    'SBY' : '1754515596',
    'SKL' : '1593027205',
    'CKR' : '573387565',
    'TBT' : '861810518'
}

bsd_df = fetch_gsheet(sheet=sheet, gid=gid['BSD'], opt=1, format_type='csv')
tjd_df = fetch_gsheet(sheet=sheet, gid=gid['TJD'], opt=1, format_type='csv')
bgr_df = fetch_gsheet(sheet=sheet, gid=gid['BGR'], opt=1, format_type='csv')
dpk_df = fetch_gsheet(sheet=sheet, gid=gid['DPK'], opt=1, format_type='csv')
klm_df = fetch_gsheet(sheet=sheet, gid=gid['KLM'], opt=1, format_type='csv')
kgd_df = fetch_gsheet(sheet=sheet, gid=gid['KGD'], opt=1, format_type='csv')
bkp_df = fetch_gsheet(sheet=sheet, gid=gid['BKP'], opt=1, format_type='csv')
bhi_df = fetch_gsheet(sheet=sheet, gid=gid['BHI'], opt=1, format_type='csv')
btr_df = fetch_gsheet(sheet=sheet, gid=gid['BTR'], opt=1, format_type='csv')
tct_df = fetch_gsheet(sheet=sheet, gid=gid['TCT'], opt=1, format_type='csv')
pjt_df = fetch_gsheet(sheet=sheet, gid=gid['PJT'], opt=1, format_type='csv')
sby_df = fetch_gsheet(sheet=sheet, gid=gid['SBY'], opt=1, format_type='csv')
skl_df = fetch_gsheet(sheet=sheet, gid=gid['SKL'], opt=1, format_type='csv')
ckr_df = fetch_gsheet(sheet=sheet, gid=gid['CKR'], opt=1, format_type='csv')
tbt_df = fetch_gsheet(sheet=sheet, gid=gid['TBT'], opt=1, format_type='csv')

/var/folders/02/c4wzpds97xv1jfyt22h304_r0000gn/T/ipykernel_4848/1181615110.py:50: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
/var/folders/02/c4wzpds97xv1jfyt22h304_r0000gn/T/ipykernel_4848/1181615110.py:50: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
/var/folders/02/c4wzpds97xv1jfyt22h304_r0000gn/T/ipykernel_4848/1181615110.py:50: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
/var/folders/02/c4wzpds97xv1jfyt22h304_r0000gn/T/ipykernel_4848/1181615110.py:50: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
/var/folders/02/c4wzpds97xv1jfyt22h304_r0000gn/T/ipykernel_4848/1181

In [6]:
query_issues(bsd_df)

/var/folders/02/c4wzpds97xv1jfyt22h304_r0000gn/T/ipykernel_4848/1605252692.py:50: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


,Date First Chat,CENTER,Pemberi Referral (no HP),Handled By


In [4]:
dfs_dict = {
    "BSD" : bsd_df,
    "TJD" : tjd_df,
    "BGR" : bgr_df,
    "DPK" : dpk_df,
    "KLM" : klm_df,
    "KGD" : kgd_df,
    "BKP" : bkp_df,
    "BHI" : bhi_df,
    "BTR" : btr_df,
    "TCT" : tct_df,
    "PJT" : pjt_df,
    "SBY" : sby_df,
    "SKL" : skl_df,
    "CKR" : ckr_df,
    "TBT" : tbt_df
}

output_path = "query_issues.xlsx"
invert_excel(dfs_dict, output_path)

/var/folders/02/c4wzpds97xv1jfyt22h304_r0000gn/T/ipykernel_13132/52562600.py:50: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
/var/folders/02/c4wzpds97xv1jfyt22h304_r0000gn/T/ipykernel_13132/52562600.py:50: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
/var/folders/02/c4wzpds97xv1jfyt22h304_r0000gn/T/ipykernel_13132/52562600.py:50: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
/var/folders/02/c4wzpds97xv1jfyt22h304_r0000gn/T/ipykernel_13132/52562600.py:50: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
/var/folders/02/c4wzpds97xv1jfyt22h304_r0000gn/T/ipykernel_13132/5256260

'query_issues.xlsx'

In [ ]:
# import gspread
# import pandas as pd
# import time
# from google.oauth2.service_account import Credentials

# # ---------------- CONFIG ----------------
# SHEET_URL = "https://docs.google.com/spreadsheets/d/1GiHAMx-K2APmNeMSq6TDgz3s8GdStWFHFYgbK9igFno/edit#gid=0"
# CREDENTIALS_FILE = "credentials.json"
# DUPLICATE_COLUMN = "Duplicate Data Checker"

# # Define which columns to check for duplicates in each sheet
# SHEET_CONFIG = {
#     'BSD': ['Kontak Referral (Nama Kids)', 'Kontak Referral (no HP)'],
#     'TJD': ['Kontak Referral (Nama Kids)', 'Kontak Referral (no HP)'],
#     'BGR': ['Kontak Referral (Nama Kids)', 'Kontak Referral (no HP)'],
#     'DPK': ['Kontak Referral (Nama Kids)', 'Kontak Referral (no HP)'],
#     'KLM': ['Kontak Referral (Nama Kids)', 'Kontak Referral (no HP)'],
#     'KGD': ['Kontak Referral (Nama Kids)', 'Kontak Referral (no HP)'],
#     'BKP': ['Kontak Referral (Nama Kids)', 'Kontak Referral (no HP)'],
#     'BHI': ['Kontak Referral (Nama Kids)', 'Kontak Referral (no HP)'],
#     'BTR': ['Kontak Referral (Nama Kids)', 'Kontak Referral (no HP)'],
#     'TCT': ['Kontak Referral (Nama Kids)', 'Kontak Referral (no HP)'],
#     'PJT': ['Kontak Referral (Nama Kids)', 'Kontak Referral (no HP)'],
#     'SBY': ['Kontak Referral (Nama Kids)', 'Kontak Referral (no HP)'],
#     'SKL': ['Kontak Referral (Nama Kids)', 'Kontak Referral (no HP)'],
#     'CKR': ['Kontak Referral (Nama Kids)', 'Kontak Referral (no HP)'],
#     'TBT': ['Kontak Referral (Nama Kids)', 'Kontak Referral (no HP)']
# }
# # ----------------------------------------

# def ensure_duplicate_header(sheet, header_name, headers):
#     """Ensure the duplicate column header exists."""
#     if header_name in headers:
#         return headers, headers.index(header_name)
#     new_col = len(headers) + 1
#     sheet.update_cell(1, new_col, header_name)
#     headers.append(header_name)
#     return headers, len(headers) - 1


# def mark_duplicates_for_sheet(sheet, duplicate_column, check_columns):
#     """Mark duplicate and unique rows safely, assuming headers are on row 2 and data starts on row 3."""
#     all_values = sheet.get_all_values()

#     # If sheet too short, skip
#     if len(all_values) < 2:
#         print(f"  Sheet '{sheet.title}' has no valid data rows, skipping.")
#         return

#     # Fixed header row and data rows
#     headers = all_values[1]     # Row 2
#     data_rows = all_values[2:]  # Row 3 onward

#     # If no data below header, skip
#     if not data_rows:
#         print(f"  Sheet '{sheet.title}' has no data rows, skipping.")
#         return

#     # Ensure duplicate column header exists
#     headers, dup_col_idx = ensure_duplicate_header(sheet, duplicate_column, headers)

#     # Build DataFrame
#     df = pd.DataFrame(data_rows, columns=headers)
#     df.columns = df.columns.str.strip()
#     df = df.dropna(how="all").reset_index(drop=True)

#     # Ensure all required columns exist
#     for col in check_columns:
#         if col not in df.columns:
#             print(f"  Warning: '{col}' not found in '{sheet.title}', skipping.")
#             return

#     # Initialize Duplicate Data Checker column
#     df[duplicate_column] = ""

#     # Identify non-empty rows in key columns
#     non_empty_mask = df[check_columns].apply(
#         lambda x: x.str.strip().replace("", pd.NA)
#     ).notna().any(axis=1)

#     # Identify duplicates among non-empty rows
#     valid_df = df.loc[non_empty_mask]
#     duplicates_mask = valid_df.duplicated(subset=check_columns, keep=False)

#     # Mark duplicates or unique values
#     df.loc[non_empty_mask, duplicate_column] = duplicates_mask.map(
#         lambda x: "Duplicate" if x else "Unique"
#     )

#     # Prepare safe update range (starts AFTER header row)
#     dup_values = df[duplicate_column].fillna("").tolist()
#     start_row = 3  # since data starts from row 3
#     end_row = start_row + len(dup_values) - 1

#     # Convert column index to letter (A, B, C, ...)
#     dup_col_letter = chr(65 + dup_col_idx)
#     update_range = f"{dup_col_letter}{start_row}:{dup_col_letter}{end_row}"

#     # Update the sheet safely (without touching the header)
#     sheet.update(update_range, [[v] for v in dup_values])

#     print(
#         f"  '{sheet.title}': rows={len(df)}, "
#         f"duplicates={df[duplicate_column].eq('Duplicate').sum()}, "
#         f"unique={df[duplicate_column].eq('Unique').sum()}, "
#         f"skipped={(~non_empty_mask).sum()}"
#     )


# def main():
#     print("Connecting to Google Sheets...")
#     scopes = [
#         'https://www.googleapis.com/auth/spreadsheets',
#         'https://www.googleapis.com/auth/drive'
#     ]
#     creds = Credentials.from_service_account_file(CREDENTIALS_FILE, scopes=scopes)
#     client = gspread.authorize(creds)
#     spreadsheet = client.open_by_url(SHEET_URL)
#     print(f"Connected to spreadsheet: {spreadsheet.title}\n")

#     last_run_rows = {}

#     while True:
#         print("\n--- Checking for updates ---")
#         for sheet_name, check_columns in SHEET_CONFIG.items():
#             try:
#                 sheet = spreadsheet.worksheet(sheet_name)
#             except gspread.exceptions.WorksheetNotFound:
#                 print(f"Sheet '{sheet_name}' not found, skipping.")
#                 continue
#             except Exception as e:
#                 print(f"Error opening sheet '{sheet_name}': {e}")
#                 continue

#             try:
#                 all_values = sheet.get_all_values()
#                 current_row_count = len(all_values)

#                 # Check if there’s a change in number of rows
#                 if sheet_name not in last_run_rows or last_run_rows[sheet_name] != current_row_count:
#                     print(f"Detected change in '{sheet_name}' (rows: {current_row_count}), updating...")
#                     mark_duplicates_for_sheet(sheet, DUPLICATE_COLUMN, check_columns)
#                     last_run_rows[sheet_name] = current_row_count
#                 else:
#                     print(f"No new data in '{sheet_name}', skipping.")
#             except Exception as e:
#                 print(f"  Unexpected error for '{sheet_name}': {e}")

#             time.sleep(0.5)  # polite delay between sheets

#         print("\nWaiting 60 seconds before next check...\n")
#         time.sleep(0.5)  # check again every 1 minute


# if __name__ == "__main__":
#     main()